In [1]:
import cv2
import csv
import numpy as np

In [2]:
filenames = []
labels = []
with open('Labels.csv') as index:
    index = csv.reader(index)
    for r in index:
        filenames.append(r[0])
        labels.append(r[1])
        
del filenames[0]
del labels[0]

In [3]:
labels_hot = []
for x in labels:
    if x == 'Positive':
        labels_hot.append([1,0,0])
    if x == 'Neutral':
        labels_hot.append([0,1,0])
    if x == 'Negative':
        labels_hot.append([0,0,1])

In [6]:
!rm -rf ./data
!mkdir data

In [7]:
lb_idx = 0
final_label=[]
f_idx = 0
classfier = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
color = (0,255,0)
for file in filenames:
    filename = './presidential_videos/' + file
    cap = cv2.VideoCapture(filename)
    count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        if not success:
            break
        gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        faceRects = classfier.detectMultiScale(gray, scaleFactor=1.5,minNeighbors=8, minSize=(32,32))
        
        if len(faceRects)>0:
            for fr in faceRects:
                x,y,w,h = fr
#                 cv2.rectangle(frame,(x-20,y-20), (x+w+10,y+h+10), color, 1)
                image = frame[y:y+h, x:x+w]
                im_name = './data/' + str(f_idx)+'.jpg'
                final_label.append(labels_hot[lb_idx])
                f_idx += 1
                cv2.imwrite(im_name, image)
                count += 1
#                 cv2.imshow('frame', frame)
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
        if count >= 15:
            break
    lb_idx += 1
#     break
# cv2.destroyAllWindows()
cap.release()

In [8]:
print(len(final_label))

14805


In [44]:
import os

image_size = 64
def resize_img(image, height = image_size, width = image_size):
    top,bottom, left, right = (0,0,0,0)
    
    h,w,_=image.shape
    
    longest_edge = max(h,w)
    
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
        pass
    
    BLACK = [0,0,0]
    constant = cv2.copyMakeBorder(image, top,bottom, left,right, cv2.BORDER_CONSTANT,value = BLACK)
    
    return cv2.resize(constant, (height,width))

In [47]:
images = []


for i in range(len(final_label)):
# for i in range(1,2):
    img_name = str(i) +'.jpg'
    image = cv2.imread('./data/'+img_name)
#     cv2.imshow('image', np.array(image,dtype=np.uint8))
#     print(image[45,45,1])
    if image is None:
        del final_label[i]
    else:
        image = resize_img(image, image_size, image_size)
#         cv2.imshow('test', np.array(image,dtype=np.uint8))
#         print(image[30,30,1])
        images.append(image)
#     break
        
print(len(images))
print(len(final_label))

14805
14805


In [36]:
# cv2.destroyAllWindows()

In [48]:
images = np.array(images, dtype='float')
final_label = np.array(final_label)

print(images.shape)
print(final_label.shape)

np.array(final_label)
np.savetxt('final_label.csv',final_label, delimiter=',',fmt = '%10.5f')

(14805, 64, 64, 3)
(14805, 3)


In [51]:
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.models import load_model
from keras import backend as K

Using TensorFlow backend.


In [64]:
import random
train_images, test_images, train_labels, test_labels = train_test_split(images, final_label, test_size = 0.3, random_state = random.randint(0,100))
print(test_labels)

[[0 1 0]
 [0 0 1]
 [0 1 0]
 ...
 [1 0 0]
 [1 0 0]
 [1 0 0]]


In [65]:
img_channels = 3

In [66]:

if K.image_data_format == 'channel_first':
    train_images = train_images.reshape(train_images.shape[0],img_channels, image_size, image_size)
    test_images = test_images.reshape(test_images.shape[0],img_channels, image_size, image_size)
    input_shape = (img_channels, image_size, image_size)
else:
    train_images = train_images.reshape(train_images.shape[0], image_size, image_size, img_channels)
    test_images = test_images.reshape(test_images.shape[0], image_size, image_size, img_channels)
    input_shape = (image_size, image_size, img_channels)

In [67]:
print(train_images.shape[0],'train samples')
print(test_images.shape[0], 'test samples')

10363 train samples
4442 test samples


In [68]:
train_images /= 255
test_images /= 255

In [82]:
class Model:
    def __init__(self):
        self.model = None
    def build_model(self, input_shape, nb_classes=3):
        self.model = Sequential()
        self.model.add(Conv2D(32, (3, 3), padding = 'same', input_shape = input_shape))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(32, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Conv2D(64, (3, 3), padding = 'same'))
        self.model.add(Activation('relu'))
        self.model.add(Conv2D(64, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.25))
        self.model.add(Flatten())
        self.model.add(Dense(512))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.25))
        self.model.add(Dense(nb_classes))
        self.model.add(Activation('softmax'))
        
    def train(self, train_images, train_labels, batch_size = 128, nb_epoch = 15, data_augmentation = True):
        self.model.compile(loss = 'categorical_crossentropy', 
                           optimizer = 'ADAM',
                           metrics = ['accuracy'])
        if not data_augmentation:
            self.model.fit(train_images, 
                           train_labels, 
                           batch_size = batch_size,
                           epochs = nb_epoch, 
                           shuffle = True)
        else:
            datagen = ImageDataGenerator(rotation_range = 20,
                                        width_shift_range = 0.2,
                                        height_shift_range = 0.2,
                                        horizontal_flip = True)
            
            self.model.fit_generator(datagen.flow(train_images, train_labels, batch_size = batch_size), epochs = nb_epoch)
            
    def evaluate(self, test_images, test_labels):
        score = self.model.evaluate(test_images, test_labels)
        print("%s:%.3f%%" % (self.model.metrics_names[1], score[1]*100))
        

In [83]:
model = Model()

In [84]:
model.build_model(input_shape,3)

In [85]:
model.train(train_images, train_labels, 128,20,True)

Epoch 1/20
81/81 [==============================] - 62s 771ms/step - loss: 1.0027 - accuracy: 0.5242
Epoch 2/20
81/81 [==============================] - 62s 770ms/step - loss: 0.9570 - accuracy: 0.5491
Epoch 3/20
81/81 [==============================] - 62s 764ms/step - loss: 0.9391 - accuracy: 0.5547
Epoch 4/20
81/81 [==============================] - 62s 769ms/step - loss: 0.9197 - accuracy: 0.5636
Epoch 5/20
81/81 [==============================] - 63s 775ms/step - loss: 0.8965 - accuracy: 0.5783
Epoch 6/20
81/81 [==============================] - 62s 760ms/step - loss: 0.8746 - accuracy: 0.5883
Epoch 7/20
81/81 [==============================] - 61s 758ms/step - loss: 0.8494 - accuracy: 0.6066
Epoch 8/20
81/81 [==============================] - 62s 761ms/step - loss: 0.8225 - accuracy: 0.6182
Epoch 9/20
81/81 [==============================] - 62s 760ms/step - loss: 0.8202 - accuracy: 0.6224
Epoch 10/20
81/81 [==============================] - 62s 760ms/step - loss: 0.8100 - accura

In [86]:
model.evaluate(test_images, test_labels)

4442/4442 [==============================] - 6s 1ms/step
accuracy:72.310%
